# Création du fichier d'entraînement complet

## Packages

In [160]:
import pandas as pd             # Traitement des tableaux de données 

## Chargement des données

Dans un premier temps, l'objectif est de récupérer toutes les valeurs de 2012 à 2022. Les données de 2010 à 2011 (inclus) comprennent des données obsolètes et peu pertinentes pour la construction du modèle. Nous éliminons donc les données des deux premières années.

Dans un second temps, il y a 4 fichiers par an. Il faudra donc récupérer les données et les "merge" afin de construire le jeu de données d'une seule année contenant toutes les informations relatives aux accidents. Cela se traduit par une relation entre le **"*Num_Acc*"** et pour certains fichiers le **"*num_veh*"**.

Ainsi, nous obtenons un fichier complet contenant toutes les colonnes possibles (par exemple, il y aura une colonne **"*secu*"**, et 3 colonnes **"*secu1*"**, **"*secu2*"** et **"*secu3*"**). Il faudra les traiter plus tard.

In [161]:
years = ['2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022']
files = ['usagers_', 'vehicules_', 'caracteristiques_', 'lieux_',]
data_path = 'Data/TRAIN/BAAC-Annee-'

full_data = pd.DataFrame()

for year in years:
    
    path_to_BAAC = data_path + year + '/'
    
    year_file = pd.DataFrame()
    
    for file_name in files:        
        path_to_file = path_to_BAAC + file_name + year + '_.csv'
        
        data = pd.read_csv(path_to_file, encoding="latin1", sep=";", low_memory=False)
        data = data.drop(data.columns[0], axis=1)
        
        if file_name == 'usagers_':
            year_file = pd.concat([year_file, data], axis=1)
        else:
            if file_name == 'vehicules_':
                year_file = pd.merge(year_file, data, on=['Num_Acc', 'num_veh'], how='left')
            else:
                year_file = pd.merge(year_file, data, on=['Num_Acc'], how='left')
    
    full_data = pd.concat([full_data, year_file], axis=0)

In [162]:
initial_row = full_data.shape[0]
initial_col = full_data.shape[1]

print(f"Le nombre d'observations initial est de : {initial_row}")
print(f"Le nombre de variables initial est de : {initial_col}")

Le nombre d'observations initial est de : 1281392
Le nombre de variables initial est de : 59


In [163]:
full_data.to_csv('Filtered_Data/TRAIN/Full_Train_Data_V0_to_V4.csv', index=False)

Désormais, nous disposons d'un **"*.csv*"** de grandes amplitudes et nous devons comprendre les données ainsi qu'harmoniser ces dernières au sein du **"*pré-traitement des données*"**. L'objectif sera de supprimer les colonnes qui ne présentent pas assez de données (voire de données pertinentes dans certains cas). Le but final est d'obtenir un jeu de données complet, prêt à être examiné pour le feature engineering, en décelant les relations sous-jacentes entre les données et éliminer les variables inutiles pour la classification.

Etant donné que l'objectif est d'obtenir un jeu de données propres réutilisables pour de futures exploitations, il n'est pas nécessaire de parser les types de variables, car ces derniers ne sont pas conservés lors de l'écriture des fichiers **".csv"**.

# Fonctions utiles

In [164]:
def delete_columns(list_col: list[str], data_frame: pd.DataFrame) -> None:
    """
    Delete the selected columns from the data frame
    :param list_col: list of columns in the dataframe to delete
    :param data_frame: dataframe where the deletion is applied
    """
    for col in list_col:
        if col in data_frame.columns:
            data_frame.drop(labels=[col], axis=1, inplace=True)
            
def process_secu(value: int) -> int:
    """
    Process the conversion from a 2 digits integer to 1 digits integer
    :param value: to be converted
    :return: only the digit associated with the security element (or 8 if nothing used)
    """
    value_str = str(value)
    if len(value_str) >= 2 and value_str[1] == '1':
        return int(value_str[0])
    else:
        return 8
    
def convert_to_int(time_str: str) -> int:
    """
    Convert the string time in an integer value to allow intervals implementation
    :param time_str: 'XX:XX', 'X:XX', 'XXX', 'X', X, XXX, ...
    :return: an integer of the time according to the time format XX(hours)XX(minutes)
    """
    if isinstance(time_str, str):
        if ':' in time_str:
            hours, minutes = map(int, time_str.split(':'))
            return hours * 100 + minutes
        else:
            return int(time_str)
    else:
        return time_str
    
def generate_intervals(start, end, step) -> list[int]:
    """
    Generate the intervals from the start to the end value by step
    :param start: first value in the interval
    :param end: last value in the interval
    :param step: range between two categories
    :return: list of intervals
    """
    list_cat = []
    for i in range(start, end, step):
        list_cat.append((i, i + step))
    return list_cat

def map_to_simple_value(value: int, list_cat: list[int]) -> int:
    """
    Function to map values to simple values based on intervals
    :param list_cat: list of the intervals
    :param value: value to map
    :return: mapped value
    """
    for i, interval in enumerate(list_cat):
        if interval[0] <= value < interval[1]:
            return simple_values[i]
    return None  

def clean_dep(dep: str) -> str:
    """
    Format the string of the department
    :param dep: French department
    :return: formatted string of the department
    """
    dep = str(dep)
    dep = dep.lstrip('0')
    if len(dep) == 3 and dep.endswith('0'):
        dep = dep[:-1]
    if dep in {'201', '202'}:
        dep = {'201': '2A', '202': '2B'}[dep]
    return dep
    
def convert_float_to_int(data_frame: pd.DataFrame, col_name: str) -> None:
    """
    Convert the column type in the related dataframe from float to int
    :param data_frame: dataframe where the column must be
    :param col_name: column to parse
    """
    data_frame[col_name] = data_frame[col_name].astype(int)

# Traitement des données relatives aux usagers

Les données relatives aux usagers contiennent en fonction des années entre 14 et 16 variables prédictives. Il faudra donc traiter ces inconsistances pour obtenir des données uniformes.

Les données à traiter sont les suivantes :
- **Num_Acc** : identifiant de l'accident. Valeur **vitale** pour joindre les données. Cependant, au-delà de la fusion des données, ce numéro n'apporte rien en matière de prédiction. En effet, il est générique et ne transmet pas d'information complémentaire. On se demande alors s'il ne vaudrait pas mieux le supprimer. Dans ce cas-là, on décide de ne pas stocker cette colonne dans l'une des listes traduisant son abandon, car on ne conservera que les variables stockées. 
- **num_veh** : de la même manière, le numéro du véhicule ne semble pas particulièrement pertinent pour la construction d'un modèle de prédiction. Etant donné qu'il n'apporte pas d'informations sur le type de véhicule, et qu'il est propre à un accident et non à un référentiel global, on préfère écarter cette situation.
- **id_vehicule** : cette variable n'est pas apparue dès le début. En effet, il a fallu attendre l'an 2019 pour la voir apparaître. Cependant, il s'avère que cette dernière correspond au numéro veh au format numérique et non alpha. On décidera donc de ne pas le retenir si elle est présente.
- **grav** : étant la colonne à prédire, il est évident que nous souhaitons conserver cette dernière. Nous nous penchons sur la qualité de cette dernière. On observe la présence de la valeur '-1' 279 fois. Proportionnellement à la taille des données, on décide donc de supprimer ces quelques lignes ne pouvant les rendre utilisables. Dans l'exercice de prédiction, il est demandé de convertir cette colonne en binaire (**'*0*'** pour les cas sans dangers [1, 4] et **'*1*'** pour les cas graves [2, 3]). Cette variable sera ainsi catégorisée en binaire conformément aux prédictions attendues.
- **sexe** : certaines études ont pu démontrer des disparités dans les comportements de conduite entre les différents sexes. Nous conservons cette donnée dans l'éventualité où une corrélation existerait entre le sexe, le statut de conducteur et la gravité. Un soupçon se porte sur l'impulsivité et la vitesse de certains conducteurs masculins.
- **an_nais** : on dispose ici de l'année de naissance et cette donnée semble pertinente. En effet, en fonction des générations, des politiques de sensibilisation plus ou moins importantes furent appliquées amenant à de potentielles disparités en matière de comportements routiers. Cependant, les victimes impliquées et les passagers potentiels risquent de biaiser l'étude. Ainsi, on décide de supprimer cette colonne.
- **trajet** : cette variable nous informe sur le motif du déplacement au moment de l'accident. Cela peut renvoyer au trajet entre le domicile et le travail, une promenade, etc. Il se pourrait qu'une corrélation existe entre des trajets récurrents et une augmentation des accidents. En effet, conformément au code de la route, il est indiqué qu'une baisse de la concentration est observée dans les trajets quotidiens.
- **locp** : permet de localiser la position du piéton ce qui est une information importante. En effet, un piéton qui traverse sur un passage piéton sans signalisation lumineuse aura plus de chance de subir des lésions graves, voire plus, qu’un piéton sur un trottoir par exemple. Cependant, en fonction des années, les valeurs peuvent changer pour ce qui est des assignations. Néanmoins, la colonne dispose de trop de variables manquantes ainsi, on supprime cette colonne.
- **etatp** : cette variable permet de préciser si le piéton accidenté était seul ou non. Il est envisageable, mais pas nécessairement vrai, que la majorité des groupes soient distraits par leurs camarades amenant à une baisse de la vigilance en matière de protection routière. On songera donc à conserver cette catégorie. Néanmoins, la colonne dispose de trop de variables manquantes ainsi, on supprime cette colonne.
- **catu** : correspond à la catégorie d'usager. Autrement dit, s'il s'agit d'un conducteur, d'un passager ou bien d'un piéton (ou encore d'un piéton en roller ou en trottinette conformément à la documentation de 2005 à 2018). Il est important de disposer de ces informations en cas de corrélations entre le statut de la victime et son état. On peut soumettre l'hypothèse que ce statut influe sur les situations dans lesquelles les usagers sont impliqués notamment (difficile d'aller à 130 km/h en tant que piéton).
- **place** : permet de situer la place occupée dans le véhicule par l'usager au moment de l'accident. Cette donnée est importante, car il existe certainement une relation entre la position des victimes et la gravité de l'accident. En effet, en fonction de la position, les dispositifs de sécurité ne sont pas les mêmes. On soupçonne donc une importante majeure de cette variable.
- **secu/secu1/secu2/secu3** : ce modèle a fortement évolué au cours du temps en passant d'un format à deux caractères indiquant dans un premier temps le système de sécurité et dans un second les conditions d'utilisation (oui/non/non déterminé). Puis dans un nouveau format à 3 colonnes (secu1/secu2/secu3), chacune des colonnes indiquait l'utilisation d'un équipement de sécurité parmi une liste d'éléments. Ainsi, la première configuration à **"*secu*"** unique sera merge avec la valeur de **"*secu1"**. Les valeurs au sein de **"*secu2*"** et **"*secu3*"** étant majoritairement vides, ces colonnes seront supprimées. Cette catégorie de variables est très intéressante. Il semble évident que des dispositifs de sécurité viennent minimiser les dégâts subits.
- **actp** : concerne l'action du piéton avant l'accident. Diffère de nouveau selon les années. Nous décidons de calquer le format sur celui possédant la meilleure précision soit le second. Cette catégorie est relativement intéressante, car elle indique si des situations peuvent amener plus ou moins fréquemment à des accidents graves. Néanmoins, la colonne dispose de trop de variables manquantes ainsi, on supprime cette colonne.

In [165]:
# Suppression des colonnes 
col_to_drop_usagers = ['Num_Acc', 'id_vehicule_x', 'id_vehicule_y', 'num_veh', 'an_nais', 'locp', 'etatp', 'secu2', 'secu3', 'actp', 'id_usager']
delete_columns(col_to_drop_usagers, full_data)

# full_data['grav']
full_data = full_data[full_data['grav'] != -1].replace({'grav': {1: 0, 4: 0, 2: 1, 3: 1}})

# full_data['sexe']
full_data = full_data[full_data['sexe'] != -1].replace({2: 0})

# full_data['trajet']
full_data = full_data.replace({'trajet': {-1: 9, 0: 9}})

# full_data['catu']
full_data.loc[full_data['catu'] == 4, 'catv'] = 99
full_data['catu'] = full_data['catu'].replace({4: 3})

# full_data['place']
full_data.loc[full_data['catu'] == 3, 'place'] = 10
full_data['place'] = full_data['place'].replace({-1: pd.NA})

# full_data['secu', 'secu1']
full_data['secu'] = full_data['secu'].apply(process_secu)
full_data['secu'] = full_data['secu'].fillna(full_data['secu1'].replace([-1, 9], 8))
delete_columns(['secu1'], full_data)

# Drop the row with empty values in the following columns
full_data= full_data.dropna(subset=['grav', 'trajet', 'sexe', 'catu', 'place', 'secu'])

Nous nous penchons sur les changements des dimensions entre les deux jeux de données (initial/post-usagers) :

In [166]:
post_usagers_row = full_data.shape[0]
post_usagers_col = full_data.shape[1]

print(f"Le nombre d'observations initial est de : {post_usagers_row} (différence de : {initial_row - post_usagers_row} lignes).")
print(f"Le nombre de variables initial est de : {post_usagers_col} (différence de : {initial_col - post_usagers_col} variables).")

Le nombre d'observations initial est de : 1275900 (différence de : 5492 lignes).
Le nombre de variables initial est de : 47 (différence de : 12 variables).


In [167]:
full_data.to_csv('Filtered_Data/TRAIN/Filtered_Train_Data_V1_to_V4.csv', index=False)

Désormais, toutes les variables propres aux usagers ont été traitées, on sauvegarde une version intermédiaire de ce fichier.

# Traitement des données relatives aux caractéristiques

Les fichiers caractéristiques contiennent globalement des informations sur le contexte de l'accident d'un point à l'aide notamment des conditions météorologiques, temporelles, etc. Certaines variables peuvent être présentes dans les fichiers originaux (particulièrement la variable 'Num_Acc'), mais absent dans le data frame traité. Cela signifie que ces variables ont déjà fait l'objet d'un traitement et il sera inutile de revenir sur ces dernières. À titre d'information, le nombre de variables à l'origine varie de 14 à 16 variables.

Les données à traiter sont les suivantes :
- **jour/mois/an/hrmn** : jour/mois/année/heure et minutes de l'accident. Ces informations sont cruciales, car des périodes pourraient être plus propices à des accidents graves. Un exemple serait qu'en hiver, une visibilité plus faible soit observée, ou bien un taux de suicide plus important pourrait expliquer une hausse de la gravité des accidents. En somme, sans fournir de justifications, ces informations peuvent être pertinentes.
- **lum** : apporte une information importante sur les conditions d'éclairage lors de l'accident. L'absence d'éclairage peu entraîner une perte de visibilité et amener à des répercussions importantes tandis qu'une meilleure visibilité permet d'éviter des drames.
- **dep/com** : numéro du département/numéro de commune répertorié à l'aide du code INSEE suivi d'un 0/comportant 3 chiffres calés à droite. Il est vrai que certaines communes peuvent être porteuses davantage que d'autres d'accidents de la route. Cependant, afin d'éviter une construction de modèle trop complexe, il est décidé d'éliminer les communes pour ne conserver que les valeurs des départements. En effet, une tendance départementale apportera suffisamment d'informations au vu des nombreuses variables disponibles.
- **agg** : valeur binaire indiquant si l'accident s'est produit au sein ou en dehors d'une agglomération. Il y a ici un indice fort intéressant, car il peut notamment traduire la vitesse des usagers. Il est donc décidé de le conserver.
- **int** : fourni des informations relatives aux différents types d'intersections. Il est intéressant de se pencher sur l'incidence des intersections en matière d'accident et si un rôle est joué dans la gravité de ces derniers.
- **atm** : cette variable fournit des informations relatives aux conditions atmosphériques lors de l'accident. Outre un travail de normalisation entre les différentes années, il est évident que les conditions climatiques impactent les performances de conduite (perte de visibilité...).
- **col** : le type de collision est fort intéressant pour pouvoir déterminer la gravité d'un accident. En effet, un véhicule percuté côté conducteur aura plus de chance de blesser gravement le conducteur que le passager à sa droite. Ainsi, une étroite corrélation pourrait apparaître entre ces variables.
- **adr** : il est difficile de composer avec cette variable tant l'hétérogénéité est importante dans les valeurs saisies. De plus, sa pertinence n'est pas démontrée quand on dispose d'ores et déjà de la latitude et de la longitude de l'accident. Cette variable sera donc supprimée.
- **lat/long/gps** : latitude de l'accident combinée à la longitude, cela pourrait en tant que variable numérique être utile pour créer des zones à accidents graves ou moins graves (carrefour en ville où de faibles accidents se produisent fréquemment par exemple). Enfin, le gps supprimé dans les dernières versions devient obsolètes quand on dispose des latitudes et longitudes. Hélas, les données de latitude et longitude sont trop peu qualitatives pour être exploitées sans un travail important. Nous décidons de supprimer ces colonnes pour l'instant. Cependant, nous nous réservons le droit de les retravailler plus tard afin d'améliorer potentiellement le modèle.

In [168]:
# Suppression des colonnes 
col_to_drop_usagers = ['com', 'adr', 'gps', 'lat', 'long']
delete_columns(col_to_drop_usagers, full_data)

# full_data['an']
full_data.loc[(full_data['an'] >= 12) & (full_data['an'] <= 18), 'an'] += 2000

# full_data['hrmn']
full_data['hrmn'] = full_data['hrmn'].apply(convert_to_int)
convert_float_to_int(full_data, 'hrmn')
intervals = generate_intervals(0, 2400, 100)
simple_values = list(range(len(intervals)))
full_data['hrmn'] = full_data['hrmn'].apply(map_to_simple_value, list_cat=intervals)

# full_data['lum']
full_data = full_data[full_data['lum'] != -1]

# full_data['agg']
full_data['agg'] = full_data['agg'].replace({2: 0})

# full_data['dep']
full_data['dep'] = full_data['dep'].apply(clean_dep)
full_data = full_data[full_data['dep'].isin(map(str, range(1, 96))) | full_data['dep'].isin(['971', '972', '974', '976'])]

# full_data['int']
full_data['int'] = full_data['int'].replace([-1, 0], pd.NA)

full_data = full_data.dropna(subset=['int', 'atm', 'col', 'an', 'hrmn', 'lum', 'agg', 'dep'])

Nous nous penchons sur les changements des dimensions entre les deux jeux de données (post-usagers/post-caractéristiques) :

In [169]:
post_caract_row = full_data.shape[0]
post_caract_col = full_data.shape[1]

print(f"Le nombre d'observations initial est de : {post_caract_row} (différence de : {post_usagers_row - post_caract_row} lignes).")
print(f"Le nombre de variables initial est de : {post_caract_col} (différence de : {post_usagers_col - post_caract_col} variables).")

Le nombre d'observations initial est de : 1088468 (différence de : 187432 lignes).
Le nombre de variables initial est de : 42 (différence de : 5 variables).


In [170]:
full_data.to_csv('Filtered_Data/TRAIN/Filtered_Train_Data_V2_to_V4.csv', index=False)

Désormais, toutes les variables propres aux caractéristiques ont été traitées, on sauvegarde une version intermédiaire de ce fichier.

# Traitement des données relatives aux lieux

Les fichiers lieux contiennent globalement des informations relatives aux voies empruntées allant de leur catégorie, à la vitesse maximale autorisée (pour certaines années), le nom de la route, etc. Certaines variables peuvent être présentes dans les fichiers originaux (particulièrement la variable 'Num_Acc'), mais absent dans le data frame traité. Cela signifie que ces variables ont déjà fait l'objet d'un traitement et il sera inutile de revenir sur ces dernières. À titre d'information, le nombre de variables à l'origine est de 18 variables avec disparité.

Les données à traiter sont les suivantes :
- **catr** : cette variable est importante, car elle indique le type de voie empruntée durant l'accident. On soupçonne qu'une corrélation forte peut exister entre les voies à grande vitesse et les accidents graves. En effet, si un accident se produit sur ce genre de voie, il y a peu de chance que ces derniers soient à faible allure. Ainsi, si les accidents sont majoritairement réalisés sous haute vitesse, il est probable que les accidents soient par conséquent plus violents.
- **voie** : il est difficile de composer avec cette variable à cause du manque de clarté autour de cette dernière. Cette variable sera donc supprimée.
- **v1/v2** : indice numérique/alphanumérique de la route. Ces variables sont trop vides pour être exploitables, ainsi, elles seront donc supprimées.
- **circ** : correspond aux différents régimes de circulation (sens unique, double sens, ...). On soupçonne un intérêt pour cette variable et nous décidons donc de la converser. Hélas, possédant trop de valeurs manquantes cette variable sera supprimée.
- **nbv** : correspond au nombre de voies, on décide de retenir cette variable, car il est possible que des voies uniques puissent être à l'origine d'accidents graves (dépassement de ligne blanche et rencontre frontale avec un autre automobiliste), ou bien vecteur d'une réduction des accidents graves (la majorité des gens peuvent ne pas vouloir franchir la ligne blanche et donc limite les dépassements dangereux).
- **vosp** : on suppose qu'il n'y a pas nécessairement de lien en l'existence d'une voie réservée, surtout lorsque cette voie n'est pas implicitement impliquée. On décide donc ne pas aborder cette variable pour l'instant. On se réserve le droit de la traiter dans des travaux futurs à des fins d'amélioration de modèles.
- **prof** : on soupçonne que le relief de la route puisse légèrement influencer la présence d'accident (manque de visibilité en sommet de côte) et nous conservons donc cette valeur sujette à délétion sous réserve qu'elle soit impertinente plus tard. Hélas, cette variable est trop vide pour être exploitable, ainsi, elle sera supprimée.
- **pr, pr1** : les points de repère ne semblent pas présenter d'intérêt au vu de ce qu'ils représentent pour les prendre en compte. Ils risqueraient de venir complexifier le modèle inutilement.
- **plan** : le tracé du plan semble important notamment pour les valeurs indiquant des virages, ou encore un tracé en **"*S"** indiquant un manque de visibilité, et des potentielles pertes d'adhérence.
- **lartpc, larrout** : on compte environ 42% de données où la largeur du terre-plein central, supposément en mètres, n'est pas indiquée. Ainsi, il est difficile de rendre cette donnée exploitable pour la construction d'un modèle prédictif. Pour ce qui est de la largeur de la route, on dénombre déjà 18% de valeurs manquantes, s'ajoute à cela des données peu qualitatives avec des valeurs en cm ou en m sans indication. De plus, certaines voies ont des rapports du simple au quintuple ce qui n'apporte pas une plus-value importante à l'analyse pour le modèle prédictif. On décide donc de supprimer ces variables pour ne pas amener notre modèle dans le faux.
- **surf** : Il est évident que les conditions liées à la nature de la chaussée amènent davantage d'accidents. On pensera notamment à des sols humides responsables du phénomène d'aquaplaning ou encore du verglas... Ainsi, nous décidons de sélectionner cette variable même si une corrélation forte risque de se produire avec d'autres variables liée aux conditions climatiques comme **"atm"**.
- **infra** : correspond à l'aménagement de la route au lieu de l'accident. Dans les cas d'aménagement, il peut y avoir une corrélation avec des accidents graves. Ainsi, on décide de conserver cette valeur.
- **situ** : la situation de l'accident (chaussée, trottoir, ...) permet d'avoir des informations intéressantes sur les circonstances de l'accident. En effet, si l'accident se produit sur un trottoir, on peut s'interroger sur la violence de l'accident pour déporter le véhicule jusqu'à l'intérieur du trottoir. Ainsi, nous décidons de retenir cette variable pour sa pertinence en assignat de manière catégorielle les différentes situations.
- **vma, env1** : la **"vma"** ou **V**itesse **M**aximale **A**utorisée est un indicateur trés intéressant, cependant le type de route spécifié au sein de la variable **"catr"** sous-entend déjà les informations relatives à ces vitesses. Seules les modifications des réglementations de vitesse ne peuvent être lues. De plus, fort de ses 65% de valeurs manquantes, c'est une variable difficilement exploitable. Il serait plus simple de l'assigner manuellement en fonction de **"catr"** ce qui la rendrait encore plus inutile. Pour ce qui est de **"env1"** qui traduit la présence d'une école à proximité, cet indicateur pourrait être intéressant malheureusement, il comporte 34% de valeurs manquantes. Au vu de la plus-value de cette variable, il est jugé non nécessaire de le retenir. On se réserve le droit d'y avoir recours ultérieurement. 

In [171]:
full_data = pd.read_csv('Filtered_Data/TRAIN/Filtered_Train_Data_V2_to_V4.csv', sep=',', low_memory=False)

In [172]:
full_data.shape

(1088468, 42)

In [173]:
# Suppression des colonnes 
col_to_drop_usagers = ['voie', 'v1', 'v2', 'vosp', 'pr', 'pr1', 'lartpc', 'larrout', 'vma', 'env1', 'circ', 'prof']
delete_columns(col_to_drop_usagers, full_data)

# full_data['nbv']
replace_dict = {'0.0': pd.NA, '1.0': '1', '2.0': '2', '3.0': '3', '4.0': '4', '5.0': '5', '6.0': '6', '7.0': '7', '8.0': '8', '9.0': '9', '10.0': '10', '11.0': '11', '12.0' :'12', '13.0' : '13', '#ERREUR' : pd.NA, -1: pd.NA, '-1.0': pd.NA, ' -1' : pd.NA}
full_data['nbv'] = full_data['nbv'].replace(replace_dict)

# full_data['plan']
full_data['plan'] = full_data['plan'].replace([-1, 0], pd.NA)

# full_data['surf']
full_data['surf'] = full_data['surf'].replace([-1, 0], pd.NA)

# full_data['infra']
full_data['infra'] = full_data['infra'].replace({-1 : pd.NA})

# full_data['situ']
full_data['situ'] = full_data['situ'].replace([-1, 0], pd.NA)

full_data = full_data.dropna(subset=['plan', 'surf', 'infra', 'situ', 'nbv'])

Nous nous penchons sur les changements des dimensions entre les deux jeux de données (post-caractéristiques/post-lieux) :

In [174]:
post_lieux_row = full_data.shape[0]
post_lieux_col = full_data.shape[1]

print(f"Le nombre d'observations initial est de : {post_lieux_row} (différence de : {post_caract_row - post_lieux_row} lignes).")
print(f"Le nombre de variables initial est de : {post_lieux_col} (différence de : {post_caract_col - post_lieux_col} variables).")

Le nombre d'observations initial est de : 688059 (différence de : 400409 lignes).
Le nombre de variables initial est de : 30 (différence de : 12 variables).


In [175]:
full_data.to_csv('Filtered_Data/TRAIN/Filtered_Train_Data_V3_to_V4.csv', index=False)

Désormais, toutes les variables propres aux usagers ont été traitées, on sauvegarde une version intermédiaire de ce fichier.

# Traitement des données relatives aux véhicules

Les fichiers véhicules contiennent globalement des informations relatives aux catégories des véhicules, aux obstacles rencontrés, aux types de chocs... Certaines variables peuvent être présentes dans les fichiers originaux (particulièrement la variable 'Num_Acc'), mais absent dans le data frame traité. Cela signifie que ces variables ont déjà fait l'objet d'un traitement et il sera inutile de revenir sur ces dernières. À titre d'information, le nombre de variables à l'origine est de 18 variables avec disparité.

Les données à traiter sont les suivantes :
- **senc** : le sens de circulation corrélé à certains endroits peut présenter un intérêt. Dans l'éventualité où cette variable serait pertinente, on décide de la retenir potentiellement pour la construction du modèle prédictif. Cependant, les données sont trop vides pour être exploitables.
- **catv** : la catégorie du véhicule peut être un vecteur important de la gravité des accidents. À bicyclette, il est certainement plus rare de faire des accidents graves que lorsqu'un poids lourd de plusieurs tonnes vient percuter un piéton ou un autre automobiliste. Dans un premier temps, on laisse la catégorisation initiale en pensant de manière candide que la segmentation des catégories de véhicules est qualitative. On se réserve le droit à l'avenir de traiter de nouveau cette catégorie pour construire un modèle plus précis ou moins sensible à l'over fitting en généralisant par exemple les poids lourds, les bus, les autocars...
- **obs, obsm** : ces deux variables sont liées aux obstacles fixes/mobiles heurtés. Ces renseignements sont primordiaux, en effet la connaissance d'un faussé peut être bien plus meurtrier au sein d'un accident (ou même un arbre), que la présence d'une glissière présente pour la sécurité.
- **choc** : le point d'impact peut certainement apporter des informations cruciales sur la gravité de l'accident. Certaines parties du véhicule peuvent amener à des conséquences plus ou moins importantes. En effet, de multiples chocs, fruits de tonneaux par exemple laisseront à coup sûr de gros dégâts au niveau du véhicule et potentiellement sur les accidentés.
- **manv** : la manœuvre précédent l'accident est trés intéressante. On peut clairement relier le fait d'aller à contresens ou bien un franchissement de terre-plein central à un accident grave, tandis qu'un accident dans le même sens dans la même file peut réduire les dégâts (percuté par un automobiliste à un "cédez le passage" par exemple).
- **motor** : indique le type de motorisation du véhicule (électrique, hybride, etc.), cependant, cette catégorie comporte 65% de valeurs manquantes, car trés récente. Néanmoins, ce n'est pas un souci, parce que la catégorie du véhicule peut apporter de nombreuses informations sous-jacentes aux types de motorisation du véhicule. Néanmoins, plus de 65% des valeurs sont vides.
- **occutc** : correspond au nombre d'occupants dans le véhicule. Il n'est pas irrecevable qu'un lien existe entre cette valeur et la gravité de l'accident. Que ce soit pour des raisons de capacité de gestion de la part des dispositifs de sécurité, ou bien des comportements des conducteurs/passagers sous grande fréquentation. À cause d'un grand nombre de valeurs manquantes, on juge la donnée inexploitable.

In [176]:
full_data = pd.read_csv('Filtered_Data/TRAIN/Filtered_Train_Data_V3_to_V4.csv', sep=',', low_memory=False)
full_data.shape

(688059, 30)

In [177]:
# Suppression des colonnes 
col_to_drop_usagers = ['senc', 'motor', 'occutc']
delete_columns(col_to_drop_usagers, full_data)

# full_data['catv']
full_data['catv'] = full_data['catv'].replace({-1: 0})

# full_data['obs']
full_data['obs'] = full_data['obs'].replace({-1: pd.NA})

# full_data['obsm']
full_data['obsm'] = full_data['obsm'].replace({-1: pd.NA})

# full_data['choc']
full_data['choc'] = full_data['choc'].replace({-1: pd.NA})

# full_data['manv']
full_data['manv'] = full_data['manv'].replace([-1, 0], pd.NA)

full_data = full_data.dropna(subset=['obs', 'obsm', 'choc', 'manv'])

Nous nous penchons sur les changements des dimensions entre les deux jeux de données (post-lieux/post-véhicules) :

In [178]:
post_vehicules_row = full_data.shape[0]
post_vehicules_col = full_data.shape[1]

print(f"Le nombre d'observations initial est de : {post_vehicules_row} (différence de : {post_lieux_row - post_vehicules_row} lignes).")
print(f"Le nombre de variables initial est de : {post_vehicules_col} (différence de : {post_lieux_col - post_vehicules_col} variables).")

Le nombre d'observations initial est de : 547774 (différence de : 140285 lignes).
Le nombre de variables initial est de : 27 (différence de : 3 variables).


In [179]:
full_data.to_csv('Filtered_Data/TRAIN/Filtered_Train_Data_V4_to_V4.csv', index=False)

Désormais, toutes les variables propres aux usagers ont été traitées, on sauvegarde une version intermédiaire de ce fichier.